# Files & imports

In [14]:
from pandas import read_csv, DataFrame
import pandas as pd

In [15]:
## https://www.kaggle.com/datasets/mkechinov/direct-messaging/data #

filename_first_purchase = "dataset/client_first_purchase_date.csv"
filename_campaigns = "dataset/campaigns.csv"

filename_msg = "dataset/messages-demo.csv"
file_tag = "ecom_msg"

data_first_purchase = pd.read_csv(filename_first_purchase)
data_campaigns = pd.read_csv(filename_campaigns)
data_msg = pd.read_csv(filename_msg)

data_first_purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854736 entries, 0 to 1854735
Data columns (total 2 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   client_id            int64 
 1   first_purchase_date  object
dtypes: int64(1), object(1)
memory usage: 28.3+ MB


# Load File

### >> method 1.1 - random sample on the merge [used]

In [16]:
import pandas as pd

# Sample each dataset
#data_first_purchase_sampled = sample_data(data_first_purchase, test_data=test)  # Replace test_data=False as needed
#data_campaigns_sampled = sample_data(data_campaigns, test_data=test)
#data_msg_sampled = sample_data(data_msg, test_data=test)

test = False 

def sample_per_day(group, fraction=0.1):
    return group.sample(frac=fraction)

if test==True: 

    sample = 0.01

    #data_first_purchase['first_purchase_date'] = pd.to_datetime(data_first_purchase['first_purchase_date']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_first_purchase = data_first_purchase.groupby('first_purchase_date').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    #data_campaigns['started_at'] = pd.to_datetime(data_campaigns['started_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_campaigns = data_campaigns.groupby('started_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    data_msg['sent_at'] = pd.to_datetime(data_msg['sent_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    data_msg = data_msg.groupby('sent_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True) 
   

else: 
    sample = 0.1

    #data_first_purchase['first_purchase_date'] = pd.to_datetime(data_first_purchase['first_purchase_date']) 
    # Apply the sampling to each group (grouped by event_date) 10%
    #data_first_purchase = data_first_purchase.groupby('first_purchase_date').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)
    
    #data_campaigns['started_at'] = pd.to_datetime(data_campaigns['started_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_campaigns = data_campaigns.groupby('started_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    data_msg['sent_at'] = pd.to_datetime(data_msg['sent_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    data_msg = data_msg.groupby('sent_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True) 

# Display some rows from the final dataset
data_first_purchase.info()
data_campaigns.info()
data_msg.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854736 entries, 0 to 1854735
Data columns (total 2 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   client_id            int64 
 1   first_purchase_date  object
dtypes: int64(1), object(1)
memory usage: 28.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1907 entries, 0 to 1906
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            1907 non-null   int64  
 1   campaign_type                 1907 non-null   object 
 2   channel                       1907 non-null   object 
 3   topic                         1877 non-null   object 
 4   started_at                    1824 non-null   object 
 5   finished_at                   1808 non-null   object 
 6   total_count                   1824 non-null   float64
 7   ab_test                       12 non-null     object 
 8   war

In [17]:
# Filter the relevant columns
first_purchase_filtered = data_first_purchase[['client_id', 'first_purchase_date']]
campaign_filtered = data_campaigns[['id', 'campaign_type', 'channel', 'topic', 'started_at',
                                            'finished_at', 'total_count', 'subject_length',
                                            'subject_with_personalization', 'subject_with_deadline',
                                            'subject_with_emoji', 'subject_with_bonuses',
                                            'subject_with_discount', 'subject_with_saleout']]

# Merge datasets using left join (or inner join if desired)
data_1 = pd.merge(data_msg, first_purchase_filtered, on='client_id', how='left')

# Create composite keys
data_1['composite_key'] = data_1['campaign_id'].astype(str) + '_' + data_1['message_type'].astype(str)
campaign_filtered['composite_key'] = campaign_filtered['id'].astype(str) + '_' + campaign_filtered['campaign_type'].astype(str)

# Perform the left join using the composite key
data_final = pd.merge(data_1, campaign_filtered, on='composite_key', how='left', suffixes=('_msg', '_cmp'))


# Ensure the columns have the same data type
#data_1['campaign_id'] = data_1['campaign_id'].astype(str)
#data_1['message_type'] = data_1['message_type'].astype(str)
#
#campaign_filtered['id'] = campaign_filtered['id'].astype(str)
#campaign_filtered['campaign_type'] = campaign_filtered['campaign_type'].astype(str)

#data_final = pd.merge(data_1, campaign_filtered, left_on=['campaign_id', "message_type"], right_on=['id', "campaign_type"], how='left', suffixes=('_msg', '_cmp'))

data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815316 entries, 0 to 815315
Data columns (total 48 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   id_msg                        815316 non-null  int64         
 1   message_id                    815316 non-null  object        
 2   campaign_id                   815316 non-null  int64         
 3   message_type                  815316 non-null  object        
 4   client_id                     815316 non-null  int64         
 5   channel_msg                   815316 non-null  object        
 6   category                      0 non-null       float64       
 7   platform                      54966 non-null   object        
 8   email_provider                392558 non-null  object        
 9   stream                        815316 non-null  object        
 10  date                          815316 non-null  object        
 11  sent_at      

In [18]:
%run "dslabs_functions.py"

dslabs_functions loaded


# Transformations

## Remove columns

### Remove IDs

In [19]:
#drop id columns - are not relevant for the prediction of the class

data_final = data_final.drop(columns=['message_id', 'id_msg', "id_cmp", 'campaign_id','client_id'])

# Drop the composite key if it's no longer needed
data_final.drop(columns=['composite_key'], inplace=True)

### Irrelevant for business

In [20]:
#drop unnecessary values to the business/problem:

data_final = data_final.drop(columns=["created_at", "updated_at", "date", "opened_last_time_at", "clicked_last_time_at",
                               "hard_bounced_at", "soft_bounced_at", "created_at", "updated_at", "finished_at"
                          ])

#created_at and updated_at, not necessary

### High MV (80%+)

In [21]:
#drop MV columns - where MV values are around 90% or above. 

data_final = data_final.drop(columns=["category", "stream", "blocked_at", "is_blocked", "message_type"
                          ])

#stream tem apenas 1 valor e nao ajuda com nada ao modelo
#category, blocked_at tem 100% MV
#is_blocked: 1 distinct values
#message_type dropped, same as campaign_type


# Split 'date' column

In [22]:
import pandas as pd

# Split the 'date'' column to have more information and detail on the:
# - day_of_month
# - day_ok_week 
# - is_weekend
# - moment_of_day
# - hour
# - min
#As the dataset only has data from sepecific month of 2021, we don't need to get the year, quarter.

# Convert the 'date' column to datetime
data_final['sent_at'] = pd.to_datetime(data_final['sent_at'], utc=True)

# Create new columns based on the date column 'event_time'
data_final['day_of_year'] = data_final['sent_at'].dt.dayofyear  # Day of the year
data_final['week_of_month'] = data_final['sent_at'].apply(lambda x: (x.day - 1) // 7 + 1)
data_final['is_weekend'] = data_final['sent_at'].dt.weekday.apply(lambda x: True if x >= 5 else False)
data_final['day_of_week_nr'] = data_final['sent_at'].dt.weekday  # Monday=0, Sunday=6
data_final['day_of_month'] = data_final['sent_at'].dt.day
data_final['month'] = data_final['sent_at'].dt.month_name()
data_final['hour'] = data_final['sent_at'].dt.hour
data_final['min'] = data_final['sent_at'].dt.minute

# Create function to get time of the day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

data_final['time_of_day'] = data_final['hour'].apply(categorize_time_of_day)

# Display the updated DataFrame
data_final.head()


,channel_msg,platform,email_provider,sent_at,is_opened,opened_first_time_at,is_clicked,clicked_first_time_at,is_unsubscribed,unsubscribed_at,...,subject_with_saleout,day_of_year,week_of_month,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day
0,email,NaN,mail.ru,2021-04-30 05:00:07+00:00,t,2021-04-30 12:46:37,f,NaN,f,NaN,...,False,120,5,False,4,30,April,5,0,Morning
1,email,NaN,mail.ru,2021-04-30 05:00:29+00:00,f,NaN,f,NaN,f,NaN,...,NaN,120,5,False,4,30,April,5,0,Morning
2,email,NaN,mail.ru,2021-04-30 05:01:34+00:00,f,NaN,f,NaN,f,NaN,...,False,120,5,False,4,30,April,5,1,Morning
3,email,NaN,yandex.ru,2021-04-30 05:01:46+00:00,f,NaN,f,NaN,f,NaN,...,NaN,120,5,False,4,30,April,5,1,Morning
4,email,NaN,yandex.ru,2021-04-30 05:02:22+00:00,t,2021-04-30 05:25:33,f,NaN,f,NaN,...,NaN,120,5,False,4,30,April,5,2,Morning


## Remove date columns
Date Variables:
-  🚫date: 20 distinct values
-  ✅sent_at: 35261 distinct values
-  ✅opened_first_time_at: 8720 distinct values
-  🚫opened_last_time_at: 8737 distinct values
-  ✅clicked_first_time_at: 892 distinct values
-  🚫clicked_last_time_at: 893 distinct values
-  ✅unsubscribed_at: 1923 distinct values
-  🚫hard_bounced_at: 4 distinct values
-  🚫soft_bounced_at: 12 distinct values
-  ✅complained_at: 19 distinct values
-  ✅purchased_at: 21 distinct values
-  🚫created_at: 71568 distinct values
-  🚫updated_at: 71568 distinct values
-  ✅first_purchase_date: 730 distinct values
-  ✅started_at: 35 distinct values - Bulk campaign start datetime
-  🚫finished_at: 35 distinct values - Bulk campaign finish datetime

Creating more variables considering "sent_at"

In [23]:
# Convert all date columns to datetime format
date_columns = ['opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
                'complained_at', 'purchased_at', 'first_purchase_date', 'started_at']

data_final[['sent_at'] + date_columns] = data_final[['sent_at'] + date_columns].apply(pd.to_datetime)

# Remove timezone information if any column has it, making all columns tz-naive
for col in ['sent_at'] + date_columns:
    data_final[col] = data_final[col].dt.tz_localize(None)

# Calculate the difference in days between 'sent_at' and each of the specified date columns
for col in date_columns:
    data_final[f'days_diff_{col}_sent_at'] = (data_final[col] - data_final['sent_at']).dt.days

# Display the updated DataFrame
data_final.head()

,channel_msg,platform,email_provider,sent_at,is_opened,opened_first_time_at,is_clicked,clicked_first_time_at,is_unsubscribed,unsubscribed_at,...,hour,min,time_of_day,days_diff_opened_first_time_at_sent_at,days_diff_clicked_first_time_at_sent_at,days_diff_unsubscribed_at_sent_at,days_diff_complained_at_sent_at,days_diff_purchased_at_sent_at,days_diff_first_purchase_date_sent_at,days_diff_started_at_sent_at
0,email,NaN,mail.ru,2021-04-30 05:00:07,t,2021-04-30 12:46:37,f,NaT,f,NaT,...,5,0,Morning,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,email,NaN,mail.ru,2021-04-30 05:00:29,f,NaT,f,NaT,f,NaT,...,5,0,Morning,NaN,NaN,NaN,NaN,NaN,337.0,NaN
2,email,NaN,mail.ru,2021-04-30 05:01:34,f,NaT,f,NaT,f,NaT,...,5,1,Morning,NaN,NaN,NaN,NaN,NaN,275.0,NaN
3,email,NaN,yandex.ru,2021-04-30 05:01:46,f,NaT,f,NaT,f,NaT,...,5,1,Morning,NaN,NaN,NaN,NaN,NaN,661.0,NaN
4,email,NaN,yandex.ru,2021-04-30 05:02:22,t,2021-04-30 05:25:33,f,NaT,f,NaT,...,5,2,Morning,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#remove other date columns:

data_final = data_final.drop(columns=['opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
                'complained_at', 'purchased_at', 'first_purchase_date', 'started_at'])


# Save df from preprocessing into a csv file

In [25]:
# Definir o caminho para salvar o arquivo CSV
filepath = r'dataset/df_msg_pre_proc.csv'

# Salvar o DataFrame em CSV
data_final.to_csv(filepath, index=False)

data_final.head()


,channel_msg,platform,email_provider,sent_at,is_opened,is_clicked,is_unsubscribed,is_hard_bounced,is_soft_bounced,is_complained,...,hour,min,time_of_day,days_diff_opened_first_time_at_sent_at,days_diff_clicked_first_time_at_sent_at,days_diff_unsubscribed_at_sent_at,days_diff_complained_at_sent_at,days_diff_purchased_at_sent_at,days_diff_first_purchase_date_sent_at,days_diff_started_at_sent_at
0,email,NaN,mail.ru,2021-04-30 05:00:07,t,f,f,f,f,f,...,5,0,Morning,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,email,NaN,mail.ru,2021-04-30 05:00:29,f,f,f,f,f,f,...,5,0,Morning,NaN,NaN,NaN,NaN,NaN,337.0,NaN
2,email,NaN,mail.ru,2021-04-30 05:01:34,f,f,f,f,f,f,...,5,1,Morning,NaN,NaN,NaN,NaN,NaN,275.0,NaN
3,email,NaN,yandex.ru,2021-04-30 05:01:46,f,f,f,f,f,f,...,5,1,Morning,NaN,NaN,NaN,NaN,NaN,661.0,NaN
4,email,NaN,yandex.ru,2021-04-30 05:02:22,t,f,f,f,f,f,...,5,2,Morning,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
data_final.shape[0]

NameError: name 'data_final' is not defined